<a href="https://colab.research.google.com/github/unverciftci/AI_Trainer/blob/main/AI_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Simple AI Trainer with Qwen3-0.6B from Scratch
# Educational step-by-step implementation

print("🚀 Welcome to AI-Driven Training!")
print("We'll build everything step by step")
print("-" * 40)

# Step 1: Install what we need
print("📦 Step 1: Installing packages...")
!pip install transformers torch matplotlib scikit-learn numpy

# Step 2: Import everything
print("📚 Step 2: Importing libraries...")
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import re

print("✅ All libraries imported successfully!")

# Step 3: Set up the AI Brain (Qwen3-0.6B)
print("\n🧠 Step 3: Loading Qwen3-0.6B AI Brain...")

class SimpleAIBrain:
    def __init__(self):
        self.model_name = "Qwen/Qwen2.5-0.5B-Instruct"  # Smaller variant that works well
        print(f"Loading {self.model_name}...")

        try:
            # Load the AI model
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16,
                device_map="auto"
            )

            # Set padding token
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            print("✅ AI Brain loaded successfully!")
            self.working = True

        except Exception as e:
            print(f"❌ AI Brain failed to load: {e}")
            print("🔄 Using simple rules instead")
            self.working = False

    def think_about_training(self, train_acc, test_acc, epoch):
        """Ask the AI what to do next"""

        if not self.working:
            # Simple backup rules
            if train_acc - test_acc > 15:
                return "reduce_learning_rate", "Too much overfitting"
            elif train_acc < 70 and epoch > 10:
                return "increase_learning_rate", "Learning too slow"
            else:
                return "continue", "Looking good"

        # Create a simple question for the AI
        question = f"""You are helping train a neural network.

Current status:
- Epoch: {epoch}
- Training accuracy: {train_acc:.1f}%
- Test accuracy: {test_acc:.1f}%

What should I do? Choose ONE:
1. continue (if training is going well)
2. reduce_learning_rate (if overfitting)
3. increase_learning_rate (if learning too slow)
4. stop_training (if good enough)

Answer:"""

        try:
            # Ask the AI
            inputs = self.tokenizer(question, return_tensors="pt", padding=True)

            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=30,
                    temperature=0.3,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )

            # Get AI response
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            ai_answer = response[len(question):].strip().lower()

            print(f"🤖 AI thinks: {ai_answer[:50]}...")

            # Extract decision from AI response
            if "reduce" in ai_answer or "lower" in ai_answer:
                return "reduce_learning_rate", "AI suggests reducing LR"
            elif "increase" in ai_answer or "faster" in ai_answer:
                return "increase_learning_rate", "AI suggests increasing LR"
            elif "stop" in ai_answer or "good enough" in ai_answer:
                return "stop_training", "AI suggests stopping"
            else:
                return "continue", "AI suggests continuing"

        except Exception as e:
            print(f"🤔 AI thinking failed: {e}")
            return "continue", "AI confused, continuing"

# Initialize our AI brain
ai_brain = SimpleAIBrain()

# Step 4: Create some data to learn from
print("\n📊 Step 4: Creating training data...")

# Make a fun moon-shaped dataset
X, y = make_moons(n_samples=800, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch format
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)

print(f"✅ Created dataset: {len(X_train)} training examples, {len(X_test)} test examples")

# Let's see our data
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='viridis', alpha=0.7)
plt.title('Training Data (Moons)')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

plt.subplot(1, 2, 2)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap='viridis', alpha=0.7)
plt.title('Test Data')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.tight_layout()
plt.show()

# Step 5: Create our neural network
print("\n🧠 Step 5: Building neural network...")

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Simple 3-layer network
        self.layer1 = nn.Linear(2, 64)    # 2 features -> 64 neurons
        self.layer2 = nn.Linear(64, 32)   # 64 -> 32 neurons
        self.layer3 = nn.Linear(32, 2)    # 32 -> 2 classes (moon shapes)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.relu(self.layer1(x))    # First layer + activation
        x = self.dropout(x)               # Prevent overfitting
        x = torch.relu(self.layer2(x))    # Second layer + activation
        x = self.layer3(x)                # Final layer (no activation)
        return x

# Create our model
model = SimpleNet()
print(f"✅ Neural network created!")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Step 6: Set up training
print("\n⚙️ Step 6: Setting up training...")

# Training setup
learning_rate = 0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

print(f"✅ Initial learning rate: {learning_rate}")
print(f"✅ Optimizer: Adam")
print(f"✅ Loss function: CrossEntropy")

# Step 7: Training functions
print("\n🔧 Step 7: Creating training functions...")

def train_one_epoch(model, X_train, y_train, optimizer, loss_function):
    """Train the model for one epoch"""
    model.train()

    # Forward pass
    predictions = model(X_train)
    loss = loss_function(predictions, y_train)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Calculate accuracy
    with torch.no_grad():
        predicted_classes = predictions.argmax(dim=1)
        accuracy = (predicted_classes == y_train).float().mean() * 100

    return loss.item(), accuracy.item()

def test_model(model, X_test, y_test, loss_function):
    """Test the model"""
    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        loss = loss_function(predictions, y_test)
        predicted_classes = predictions.argmax(dim=1)
        accuracy = (predicted_classes == y_test).float().mean() * 100

    return loss.item(), accuracy.item()

print("✅ Training functions ready!")

# Step 8: The main AI-driven training loop
print("\n🚀 Step 8: Starting AI-driven training!")
print("The AI will make decisions every 5 epochs")
print("-" * 50)

# Storage for results
train_accuracies = []
test_accuracies = []
losses = []
learning_rates = []
ai_decisions = []

max_epochs = 50
current_lr = learning_rate

for epoch in range(max_epochs):
    # Train for one epoch
    train_loss, train_acc = train_one_epoch(model, X_train, y_train, optimizer, loss_function)
    test_loss, test_acc = test_model(model, X_test, y_test, loss_function)

    # Store results
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    losses.append(train_loss)
    learning_rates.append(current_lr)

    # Print progress
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1:2d}: Train {train_acc:5.1f}% | Test {test_acc:5.1f}% | Loss {train_loss:.4f}")

    # Let AI make decisions every 5 epochs
    if (epoch + 1) % 5 == 0 and epoch < max_epochs - 1:
        print("\n🤖 AI is thinking...")

        action, reason = ai_brain.think_about_training(train_acc, test_acc, epoch + 1)
        ai_decisions.append((epoch + 1, action, reason))

        print(f"🧠 Decision: {action}")
        print(f"💭 Reason: {reason}")

        # Apply AI decision
        if action == "reduce_learning_rate":
            current_lr = current_lr * 0.7  # Reduce by 30%
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_lr
            print(f"📉 Learning rate → {current_lr:.6f}")

        elif action == "increase_learning_rate":
            current_lr = current_lr * 1.3  # Increase by 30%
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_lr
            print(f"📈 Learning rate → {current_lr:.6f}")

        elif action == "stop_training":
            print("🛑 AI decided to stop training early!")
            break

        print("-" * 30)

print(f"\n🎉 Training completed!")
print(f"🏆 Final test accuracy: {test_accuracies[-1]:.1f}%")

# Step 9: Show results
print("\n📊 Step 9: Visualizing results...")

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Accuracy over time
epochs = range(1, len(train_accuracies) + 1)
ax1.plot(epochs, train_accuracies, 'b-', label='Training', linewidth=2)
ax1.plot(epochs, test_accuracies, 'r-', label='Test', linewidth=2)
ax1.set_title('🎯 Accuracy Progress')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy (%)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Loss over time
ax2.plot(epochs, losses, 'g-', linewidth=2)
ax2.set_title('📉 Training Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.grid(True, alpha=0.3)

# Plot 3: Learning rate changes
ax3.plot(epochs, learning_rates, 'purple', linewidth=2, marker='o', markersize=3)
ax3.set_title('🤖 AI Learning Rate Decisions')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Learning Rate')
ax3.set_yscale('log')
ax3.grid(True, alpha=0.3)

# Plot 4: AI decisions summary
ax4.axis('off')
ax4.set_title('🧠 AI Decisions Made', fontsize=14, weight='bold')

decision_text = ""
for epoch, action, reason in ai_decisions[-4:]:  # Show last 4 decisions
    emoji = {"continue": "➡️", "reduce_learning_rate": "📉",
             "increase_learning_rate": "📈", "stop_training": "🛑"}.get(action, "🤔")
    decision_text += f"Epoch {epoch}: {emoji} {action}\n{reason}\n\n"

ax4.text(0.05, 0.95, decision_text, fontsize=10, transform=ax4.transAxes,
         verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.show()

# Step 10: Final summary
print("\n" + "="*50)
print("🎊 TRAINING COMPLETE!")
print("="*50)
print(f"📈 Best accuracy: {max(test_accuracies):.1f}%")
print(f"🤖 AI decisions made: {len(ai_decisions)}")
print(f"📉 Final learning rate: {current_lr:.6f}")
print(f"🔄 Started with LR: {learning_rate:.6f}")

if ai_decisions:
    print(f"\n🧠 AI Decision Summary:")
    action_counts = {}
    for _, action, _ in ai_decisions:
        action_counts[action] = action_counts.get(action, 0) + 1

    for action, count in action_counts.items():
        emoji = {"continue": "➡️", "reduce_learning_rate": "📉",
                "increase_learning_rate": "📈", "stop_training": "🛑"}.get(action, "🤔")
        print(f"  {emoji} {action}: {count} times")

print("\n🎓 What you learned:")
print("- How to load and use a small LLM (Qwen3-0.6B)")
print("- How to create a neural network from scratch")
print("- How to let AI make training decisions")
print("- How to visualize training progress")
print("="*50)

print("\n✨ Try changing the dataset to 'circles' or 'linear' and see how the AI adapts!")